In [1]:
import pandas as pd
import glob2
import matplotlib.pyplot as plt
import datetime
import re
from datetime import datetime, timedelta
%matplotlib inline

In [2]:
global_music_file_paths = glob2.glob('spotify za/*.csv')

In [3]:
print(len(global_music_file_paths))

214


In [4]:
print(global_music_file_paths[:4])

['spotify za\\regional-za-weekly-2018-03-09--2018-03-16.csv', 'spotify za\\regional-za-weekly-2018-03-16--2018-03-23.csv', 'spotify za\\regional-za-weekly-2018-03-23--2018-03-30.csv', 'spotify za\\regional-za-weekly-2018-03-30--2018-04-06.csv']


In [5]:
# global_music_file_paths

In [6]:
#Add the dates to the files - run once
def addsDatesToData():
    to_remove_front = "spotify za\\regional-za-weekly-"
    to_remove_back = ".csv"

    len_front_remove = len(to_remove_front)
    len_back_remove = len(to_remove_back)

    for file in global_music_file_paths: 
#         print(file)
        the_data_in_file = pd.read_csv(file)
#         the_data_in_file = the_data_in_file.iloc[1: , :]
#         the_data_in_file.columns = ["Position", "Track Name", "Artist", "Streams", "URL"]
        
        
        the_dates = file[len_front_remove:]
        the_dates = the_dates[:-len_back_remove]

        split_dates = the_dates.split("--")

        start_date = split_dates[0]
        end_date = split_dates[1]

        len_dates = the_data_in_file.shape[0]
        start_dates = len_dates * [start_date]
        end_dates = len_dates * [end_date]

        date_columns = {"Start Date": start_dates, "End Date": end_dates}   

        the_data_in_file["Start Date"] = start_dates
        the_data_in_file["End Date"] = end_dates

        #write the data back to the File
        the_data_in_file.to_csv(file, index=False)
        
#remove dates in existing files
def removeExistingDates():
    for file in global_music_file_paths:
        
        the_data_in_file = pd.read_csv(file)

        the_data_in_file.drop(["Start Date", "End Date"], axis=1, inplace=True) 

        the_data_in_file.to_csv(file, index=False)
    
#check for dates ... opting for removal and putting them back on


In [7]:
# all_files_dates = pd.DataFrame()
# def findmissingDates():
#     all_files_dates = pd.read_csv("all_files.csv", encoding="utf-8")
    
#     all_files_dates = all_files_dates[["Start Date", "End Date"]] 
    
#     all_files_dates = all_files_dates.drop_duplicates().reset_index().drop(["index"], axis=1)
    
#     return all_files_dates   

In [8]:
# all_files_dates = findmissingDates()

In [9]:
# all_files_dates

In [10]:
# removeExistingDates()
addsDatesToData()

In [11]:
# the_data_in_file

In [12]:
# pd.date_range(start ='2019-11-01', end = '2019-12-27').difference(all_files_dates.index)

In [13]:
#Combine the files and sort by dates
all_files = []
trouble_files = []
for file in global_music_file_paths:
    the_data_in_file = pd.read_csv(file)
    
    if("URL" in list(the_data_in_file.columns)):
    
        the_data_in_file = the_data_in_file.drop(["URL"], axis=1)
    
    list_vals = sum(list(the_data_in_file.isnull().sum()))
    
    if(list_vals > 0):
        trouble_files.append(file)
    all_files.append(the_data_in_file)
    
the_spotify_data = pd.concat(all_files) 
# trouble_files_df = pd.concat(trouble_files)

In [14]:
# the_spotify_data["Start Date"].unique().tolist()

In [15]:
all_cols = list(the_spotify_data.columns)
drop_cols = []

for col in all_cols:
    if("Unnamed" in col or "artificial" in col):
        drop_cols.append(col)
        
the_spotify_data.drop(drop_cols, axis=1, inplace = True)

In [16]:
the_spotify_data.columns

Index(['Position', 'Track Name', 'Artist', 'Streams', 'Start Date',
       'End Date'],
      dtype='object')

In [17]:
the_spotify_data

,Position,Track Name,Artist,Streams,Start Date,End Date
0,1,God's Plan,Drake,10313,2018-03-09,2018-03-16
1,2,Psycho (feat. Ty Dolla $ign),Post Malone,6588,2018-03-09,2018-03-16
2,3,rockstar,Post Malone,5738,2018-03-09,2018-03-16
3,4,All The Stars (with SZA),Kendrick Lamar,5585,2018-03-09,2018-03-16
4,5,Love Lies (with Normani),Khalid,5535,2018-03-09,2018-03-16
...,...,...,...,...,...,...
195,196,Better,Khalid,25185,2022-05-20,2022-05-27
196,197,Tekkies Brand,Irene-Louise Van Wyk,25143,2022-05-20,2022-05-27
197,198,Could've Been (feat. Bryson Tiller),H.E.R.,25068,2022-05-20,2022-05-27
198,199,Beautiful Mistakes (feat. Megan Thee Stallion),Maroon 5,25006,2022-05-20,2022-05-27


In [18]:
# the_spotify_data["End Date"].unique().tolist()

In [19]:
the_spotify_data["Week"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data["Year"] = the_spotify_data["Week"]

the_spotify_data['Week'] = the_spotify_data['Week'].apply(lambda x : x.isocalendar()[1] )
the_spotify_data['Year'] = the_spotify_data['Year'].apply(lambda x : x.isocalendar()[0] )

the_spotify_data["Start Date Dt"] = the_spotify_data["Start Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data["End Date Dt"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data['Seconds since Epoch'] = the_spotify_data['End Date Dt'].apply(lambda x : round(x.timestamp(), 0))

In [20]:
#get the month when this happened
def changedaytotheFirst(x):
    x = x[:-2] + "01"
    
    return x

In [21]:
the_spotify_data["End Date Aggregated"] = the_spotify_data["End Date"].apply(lambda x : changedaytotheFirst(x) )

In [22]:
#find the number of artists on the track
the_spotify_data.dropna(inplace=True)

the_spotify_data["Artist"] = the_spotify_data["Artist"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["Artist"] = the_spotify_data["Artist"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["Artist"] = the_spotify_data.apply(lambda x : x["Artist"] + "Artist_NA" if(x["Artist"] == "") else x["Artist"], axis=1)

the_spotify_data["Track Name"] = the_spotify_data["Track Name"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["Track Name"] = the_spotify_data["Track Name"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["Track Name"] = the_spotify_data.apply(lambda x : x["Artist"] + "_NA" if(x["Track Name"] == "") else x["Track Name"], axis=1)

def countNumberArtists(x):
    len_all_featured_artists = 0
    num_artists = 0
    xvals = x.split("feat")
    len_xvals = len(xvals)
    
    if(len_xvals > 1):
        featured_artists = xvals[1:]
        featured_artists_str = ' '.join(featured_artists)
        all_featured_artists = featured_artists_str.split("& ")
        len_all_featured_artists = len(all_featured_artists)
        
    num_artists = 1 + len_all_featured_artists
    
    return num_artists
        
    
the_spotify_data['ArtistCount'] = the_spotify_data['Track Name'].apply(lambda x : countNumberArtists(x))

In [23]:
the_spotify_data.shape

(42686, 13)

In [24]:
the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["index"], axis=1, inplace=True)

In [25]:
the_spotify_data.shape

(42686, 13)

In [26]:
def everInTopTen(x):
    position_vals = list(x["Position"])
    position_vals = list(set(position_vals))
    
#     print(position_vals)
    
    min_pos = min(position_vals)
    existence_val = 0
    
    if(min_pos <= 10):
        existence_val = 1
    
    x["isTopTen"] = [existence_val] * len(x)
    
    return x
    
the_spotify_data = the_spotify_data[~the_spotify_data.index.duplicated()]
the_spotify_data = the_spotify_data.groupby(["Artist", "Track Name"]).apply(lambda x : everInTopTen(x))

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [27]:
the_spotify_data.head()

,Position,Track Name,Artist,Streams,Start Date,End Date,Week,Year,Start Date Dt,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,isTopTen
0,1,God's Plan,Drake,10313,2018-03-09,2018-03-16,11,2018,2018-03-09,2018-03-16,1.521158e+09,2018-03-01,1,1
1,2,Psycho (feat. Ty Dolla $ign),Post Malone,6588,2018-03-09,2018-03-16,11,2018,2018-03-09,2018-03-16,1.521158e+09,2018-03-01,2,1
2,3,rockstar,Post Malone,5738,2018-03-09,2018-03-16,11,2018,2018-03-09,2018-03-16,1.521158e+09,2018-03-01,1,1
3,4,All The Stars (with SZA),Kendrick Lamar,5585,2018-03-09,2018-03-16,11,2018,2018-03-09,2018-03-16,1.521158e+09,2018-03-01,1,1
4,5,Love Lies (with Normani),Khalid,5535,2018-03-09,2018-03-16,11,2018,2018-03-09,2018-03-16,1.521158e+09,2018-03-01,1,1


### When did the track reach the Top 10 

### how long does a song last on the charts?

In [28]:
def lengthSong(x):
    min_date = min(x["Start Date"])
    max_date = max(x["End Date"])
    
    def convertStrToDate(y):
        split_date = y.split("-")
        dateindtformat = datetime(int(split_date[0]), int(split_date[1]), int(split_date[2]))
        
        return dateindtformat
    
    x["lenOnCharts"] = [int(((convertStrToDate(max_date) - convertStrToDate(min_date)).total_seconds() / 604800)) ] * len(x)
    x["lenOnCharts"] = x["lenOnCharts"]
    
    return x
    
the_spotify_data = the_spotify_data.groupby(["Artist", "Track Name"], as_index=False).apply(lambda x : lengthSong(x))

In [29]:
the_spotify_data.head()

,Position,Track Name,Artist,Streams,Start Date,End Date,Week,Year,Start Date Dt,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,isTopTen,lenOnCharts
0,1,God's Plan,Drake,10313,2018-03-09,2018-03-16,11,2018,2018-03-09,2018-03-16,1.521158e+09,2018-03-01,1,1,113
1,2,Psycho (feat. Ty Dolla $ign),Post Malone,6588,2018-03-09,2018-03-16,11,2018,2018-03-09,2018-03-16,1.521158e+09,2018-03-01,2,1,78
2,3,rockstar,Post Malone,5738,2018-03-09,2018-03-16,11,2018,2018-03-09,2018-03-16,1.521158e+09,2018-03-01,1,1,7
3,4,All The Stars (with SZA),Kendrick Lamar,5585,2018-03-09,2018-03-16,11,2018,2018-03-09,2018-03-16,1.521158e+09,2018-03-01,1,1,219
4,5,Love Lies (with Normani),Khalid,5535,2018-03-09,2018-03-16,11,2018,2018-03-09,2018-03-16,1.521158e+09,2018-03-01,1,1,68


In [30]:
# the_spotify_data["lenOnCharts"].unique().tolist()

In [31]:
# def getWeeksOnChart(x):
#     min_top_ten_date = min(x["Start Date"])
    
#     x["firstWeekOnChartTopTen"] = x.apply(lambda x : True if(x["isTopTen"] == 1 and x["Start Date"] == min_top_ten_date) else False , axis=1)    
    
#     def convertStrToDate(y):
#         split_date = y.split("-")
#         dateindtformat = datetime(int(split_date[0]), int(split_date[1]), int(split_date[2]))
        
#         return dateindtformat
        
#     firstweekdate = x[x["firstWeekOnChartTopTen"] == True]["Start Date"].unique().tolist()
#     if(len(firstweekdate) == 0):
#         firstweekdate = x["Start Date"].unique().tolist()
        
#     firstweekdate = firstweekdate[0]
    
   
# GET WEEK ON CHART
#     x["weeksOnChartToTopTen"] = x.apply(lambda art : convertStrToDate(firstweekdate) - convertStrToDate(min_top_ten_date), axis=1)
    
#     return x         
    
# the_spotify_data =the_spotify_data.groupby(["Artist", "Track Name"], as_index=False).apply(lambda x : getWeeksOnChart(x))

In [32]:
# the_spotify_data_top_ten = the_spotify_data_top_ten.reset_index()

In [33]:
# the_spotify_data_top_ten.columns

In [34]:
# the_spotify_data_top_ten.drop(['index'], axis=1, inplace=True)
# the_spotify_data_top_ten.head()

In [35]:
# the_spotify_data_top_ten.columns = ["Artist", "Track Name", "isTopTen"]

In [36]:
# the_spotify_data.shape, the_spotify_data.shape

In [37]:
# the_spotify_data_new = the_spotify_data.merge(the_spotify_data_top_ten, left_on=["Artist", "Track Name"], right_on=["Artist", "Track Name"], how="outer")

In [38]:
# the_spotify_data.shape, the_spotify_data_top_ten.shape

In [39]:
# the_spotify_data = the_spotify_data_top_ten

In [40]:
the_spotify_data["isTopTen"].value_counts()

0    28607
1    14079
Name: isTopTen, dtype: int64

In [41]:
the_spotify_data.isnull().sum()

Position               0
Track Name             0
Artist                 0
Streams                0
Start Date             0
End Date               0
Week                   0
Year                   0
Start Date Dt          0
End Date Dt            0
Seconds since Epoch    0
End Date Aggregated    0
ArtistCount            0
isTopTen               0
lenOnCharts            0
dtype: int64

In [42]:
unique_artists = the_spotify_data["Artist"].unique().tolist()
num_artists = len(unique_artists)
print(num_artists)

864


In [43]:
### find new artists in that week

In [44]:
# the_spotify_data.head(12)
def getArtistAppearanceCount(x):
    
    x = x.reset_index()
    x.drop(["index"],axis=1,inplace=True)
    x = x.reset_index()
    
    x_cols = x.columns
    
    x_cols = [w.replace('index', 'artistAppearanceCount') for w in x_cols]
    
    x.columns = x_cols
    
    return x        
    
the_spotify_data =the_spotify_data.groupby(["Artist"], as_index=False).apply(lambda x : getArtistAppearanceCount(x))
the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["level_0", "level_1"], inplace=True, axis=1)

In [45]:
the_spotify_data.head()

,artistAppearanceCount,Position,Track Name,Artist,Streams,Start Date,End Date,Week,Year,Start Date Dt,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,isTopTen,lenOnCharts
0,0,194,Doja,$NOT,23789,2022-02-04,2022-02-11,6,2022,2022-02-04,2022-02-11,1.644538e+09,2022-02-01,1,0,1
1,0,182,"...And to Those I Love, Thanks for Sticking Ar...",$uicideboy$,23341,2021-10-01,2021-10-08,40,2021,2021-10-01,2021-10-08,1.633651e+09,2021-10-01,1,0,1
2,0,78,Bigger Than You (feat. Drake & Quavo),2 Chainz,5994,2018-06-15,2018-06-22,25,2018,2018-06-15,2018-06-22,1.529626e+09,2018-06-01,3,0,7
3,1,90,Bigger Than You (feat. Drake & Quavo),2 Chainz,5265,2018-06-22,2018-06-29,26,2018,2018-06-22,2018-06-29,1.530230e+09,2018-06-01,3,0,7
4,2,146,Bigger Than You (feat. Drake & Quavo),2 Chainz,3964,2018-06-29,2018-07-06,27,2018,2018-06-29,2018-07-06,1.530835e+09,2018-07-01,3,0,7


In [46]:
def getTrackAppearanceCount(x):
    
    x = x.reset_index()
    x.drop(["index"],axis=1,inplace=True)
    x = x.reset_index()
    
    x_cols = x.columns
    
    x_cols = [w.replace('index', 'trackAppearanceCount') for w in x_cols]
    
    x.columns = x_cols
    
    return x         
    
the_spotify_data =the_spotify_data.groupby(["Artist", "Track Name"], as_index=False).apply(lambda x : getTrackAppearanceCount(x))
the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["level_0", "level_1"], inplace=True, axis=1)

In [47]:
the_spotify_data.head()

,trackAppearanceCount,artistAppearanceCount,Position,Track Name,Artist,Streams,Start Date,End Date,Week,Year,Start Date Dt,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,isTopTen,lenOnCharts
0,0,0,194,Doja,$NOT,23789,2022-02-04,2022-02-11,6,2022,2022-02-04,2022-02-11,1.644538e+09,2022-02-01,1,0,1
1,0,0,182,"...And to Those I Love, Thanks for Sticking Ar...",$uicideboy$,23341,2021-10-01,2021-10-08,40,2021,2021-10-01,2021-10-08,1.633651e+09,2021-10-01,1,0,1
2,0,0,78,Bigger Than You (feat. Drake & Quavo),2 Chainz,5994,2018-06-15,2018-06-22,25,2018,2018-06-15,2018-06-22,1.529626e+09,2018-06-01,3,0,7
3,1,1,90,Bigger Than You (feat. Drake & Quavo),2 Chainz,5265,2018-06-22,2018-06-29,26,2018,2018-06-22,2018-06-29,1.530230e+09,2018-06-01,3,0,7
4,2,2,146,Bigger Than You (feat. Drake & Quavo),2 Chainz,3964,2018-06-29,2018-07-06,27,2018,2018-06-29,2018-07-06,1.530835e+09,2018-07-01,3,0,7


In [48]:
# artists = the_spotify_data["Artist"]
# the_spotify_data.drop(["Artist"],axis=1,inplace=True)
# the_spotify_data.reset_index(inplace=True)
# the_spotify_data.drop(["level_1"], axis=1, inplace=True)
# the_spotify_data["Artist"] = list(artists)

In [49]:
# the_spotify_data.drop(['index'], inplace=True, axis=1)

In [50]:
the_spotify_data[["Position", "Start Date", "End Date"]].to_csv("the_spotify_data_val.csv", index=False)

In [51]:
the_spotify_data.to_csv("the_spotify_data.csv")

In [52]:
the_spotify_data["Position"] = the_spotify_data["Position"].astype(int)
the_spotify_data["Streams"] = the_spotify_data["Streams"].astype(int)
the_spotify_data["ArtistCount"] = the_spotify_data["ArtistCount"].astype(int)

the_spotify_data["isTopTen"] = the_spotify_data["isTopTen"].astype(int)
the_spotify_data["lenOnCharts"] = the_spotify_data["lenOnCharts"].astype(int)
the_spotify_data["Year"] = the_spotify_data["Year"].astype(int)
the_spotify_data["Week"] = the_spotify_data["Week"].astype(int)

In [53]:
all_cols = list(the_spotify_data.columns)
drop_cols = []

for col in all_cols:
    if("Unnamed" in col or "artificial" in col):
        drop_cols.append(col)
        
the_spotify_data.drop(drop_cols, axis=1, inplace = True)

In [54]:
the_spotify_data["Artist and Track"] = the_spotify_data["Artist"] + "; " + the_spotify_data["Track Name"]

In [55]:
# the_spotify_data.to_csv("Classification/all_files.csv", index=False)

### Getting the list of Artists and Tracks

In [56]:
import pandas as pd
import pymongo
import getlyrics

In [57]:
def trackAppearance(x):
#     x = x.reset_index()

    x = x.sort_values(by=["Start Date"])
#     x.set_index(["Position"])

    x["Position"] = x["Position"].astype(int)
#     print(list(x["Position"]))
    
    x["Position Difference"] = x["Position"].diff().fillna(0)
    x["Position Difference"] = x["Position Difference"].astype(int)
    
#     x["Position Difference"] = x["Position"].sub(x["Position"].shift())
#     x.drop(["index"], inplace=True)
    
    return x
    
all_files_grouped = the_spotify_data.groupby(["Artist", "Track Name"]).apply(lambda x: trackAppearance(x))

In [58]:
all_files_grouped["Position Difference"].max()

184

### Position over time

In [59]:
def positionvertime(x):
    posovertime = 0
    #new track
    if(x["Position Difference"] == 0 and x["trackAppearanceCount"]==0):
        posovertime = 0
    #track stayed in the same position
    elif(x["Position Difference"] == 0 and x["trackAppearanceCount"]!=0):
        posovertime = 50
    #track went up the chart
    elif(x["Position Difference"] < 0):
        posovertime = 75
    #track fell down the chart
    else:
        posovertime = 100
    return posovertime

In [60]:
all_files_grouped["Position over Time"] = all_files_grouped.apply(lambda x : positionvertime(x), axis=1)

In [61]:
all_files_grouped.to_csv("Classification/all_files_ZA.csv", index=False)

In [62]:
all_files_grouped.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'Position',
       'Track Name', 'Artist', 'Streams', 'Start Date', 'End Date', 'Week',
       'Year', 'Start Date Dt', 'End Date Dt', 'Seconds since Epoch',
       'End Date Aggregated', 'ArtistCount', 'isTopTen', 'lenOnCharts',
       'Artist and Track', 'Position Difference', 'Position over Time'],
      dtype='object')

In [63]:
# new_all_files_grouped = all_files_grouped.reset_index(drop=True)

In [64]:
def newstuff(x):
#     x = x.reset_index()

    x = x.sort_values(by=['Seconds since Epoch'])
#     x.set_index(["Position"])
    print(x)
    
    return x
    


In [65]:
# all_files_grouped = new_all_files_grouped.groupby(["Artist", "Track Name"]).apply(lambda x: newstuff(x))

In [66]:
# all_files_grouped.min()

In [67]:
# df = pd.DataFrame({'A': 'a a b'.split(), 'B': [1,2,3], 'C': [4,6, 5]})

In [68]:
# new_all_files_grouped.groupby([])

In [69]:
# all_files_grouped.apply(lambda x : print(type(x)) )

In [70]:
# # new_all_files_grouped.h'Track Name', 'Artist'ead(5)
# cols_ = ['Start Date', 'Seconds since Epoch', 'Position']
# new_test_df = pd.DataFrame()

In [71]:
# 604800
def weekonweekposition(x):
    
#     print(x.values)
    
    pd.DataFrame({'email':x.index, 'list':x.values})
    
    
#     return x_
    
#     x_ = pd.DataFrame(x, columns=['Start Date'])
    
#     x_ = x.to_frame()
    
#     print(x_.columns)
    
#     for n in range(0, fin_len):
#         for col in cols_:
#             print(len(x[n]), x[n], col)
        
    
#     x_ = pd.DataFrame(x, columns=cols_)
#     print(x_.shape)
#     print(x.columns)

In [72]:
# all_files_gr = new_all_files_grouped[['Artist', 'Track Name', 'Seconds since Epoch', 'Position']]

In [73]:
# all_files_grouped[cols_].apply( lambda x : weekonweekposition(x) )

In [74]:
# new_test_df

### South African Artists

In [75]:
all_files_grouped.shape

(42686, 20)

In [76]:
all_files_grouped.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'Position',
       'Track Name', 'Artist', 'Streams', 'Start Date', 'End Date', 'Week',
       'Year', 'Start Date Dt', 'End Date Dt', 'Seconds since Epoch',
       'End Date Aggregated', 'ArtistCount', 'isTopTen', 'lenOnCharts',
       'Artist and Track', 'Position Difference', 'Position over Time'],
      dtype='object')

In [77]:
narrowing_df = all_files_grouped[['End Date', "Position Difference"]]

In [78]:
narrowing_df.reset_index(inplace=True)

In [79]:
narrowing_df.drop(["level_2"], axis=1, inplace=True)

C:\Users\lkhum\anaconda3\envs\tfenviron\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [80]:
narrowing_df = narrowing_df[narrowing_df["Position Difference"] == 0]

In [81]:
narrowing_df.to_csv("narrow_uniq.csv", index=False)

In [82]:
# narow_artists_unique

In [83]:
uniquartists_cat_df = pd.read_csv("Classification/uniquartists_cat.csv")

In [84]:
narrowing_df_art = narrowing_df[["Artist"]]
uniquartists_cat_df_art = uniquartists_cat_df[["Artist"]]

narrowing_df_art.drop_duplicates(inplace=True)
uniquartists_cat_df_art.drop_duplicates(inplace=True)

<ipython-input-84-175321728619>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  narrowing_df_art.drop_duplicates(inplace=True)
<ipython-input-84-175321728619>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniquartists_cat_df_art.drop_duplicates(inplace=True)


In [85]:
len(narrowing_df_art), len(uniquartists_cat_df_art)

(864, 864)

In [86]:
minus_list = list(set(narrowing_df_art["Artist"]) - set(uniquartists_cat_df_art["Artist"]))

In [87]:
minus_list_df = pd.DataFrame()
minus_list_df["Artist"] = minus_list
minus_list_df["Country"] = [""]*len(minus_list)
minus_list_df.to_csv("Classification/New Artist to Cat.csv", index=False)

In [88]:
edited_minus_list_df = pd.read_csv("Classification/New Artist to Cat.csv")
uniquartists_cat_df = pd.read_csv("Classification/uniquartists_cat.csv")

added_files = [edited_minus_list_df, uniquartists_cat_df]
added_files_df = pd.concat(added_files)
added_files_df.to_csv("Classification/uniquartists_cat.csv", index=False)

In [89]:
south_african_artists_df = added_files_df[added_files_df["Country"] == "ZA"]
south_african_artists = south_african_artists_df["Artist"].tolist()

In [90]:
len(south_african_artists)

236

In [91]:
south_african_artists_lower = []
for sartists in south_african_artists:
    south_african_artists_lower.append(sartists.lower())

In [92]:
uniq_sans = south_african_artists_lower

In [93]:
len(uniq_sans)

236

In [94]:
len(south_african_artists)

236

In [95]:
all_files_grouped['Lower Artist'] = all_files_grouped['Artist'].apply(lambda x : x.lower() )

In [96]:
boolean_series = all_files_grouped["Lower Artist"].isin(uniq_sans)
all_files_grouped_filtered_df = all_files_grouped[boolean_series]

In [97]:
# df_sa = pd.read_csv("SA Musicians List.csv")

In [98]:
# sa_artists_scrapped = df_sa["Artist"].unique().tolist()

In [99]:
# sa_artists_scrapped_lower = []
# for uniqart in sa_artists_scrapped:
#     sa_artists_scrapped_lower.append(uniqart.lower())

In [100]:
# non_uniq = list(set(sa_artists_scrapped_lower) - set(south_african_artists_lower) )

# uniq_sans = list(set(sa_artists_scrapped_lower) - set(non_uniq) )

In [101]:
# len(uniq_sans)

In [102]:
all_files_grouped_filtered_df.to_csv("Classification/JustSouthAfricansOnZAChart.csv", index=False)

In [103]:
all_files_grouped_filtered_df.drop(["Artist", "Track Name"], inplace=True, axis=1)

C:\Users\lkhum\anaconda3\envs\tfenviron\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [104]:
all_files_grouped_filtered_df = all_files_grouped_filtered_df.reset_index()

In [105]:
all_files_grouped_filtered_df.drop(["level_2"], axis=1, inplace = True)

In [106]:
# treemap_df = all_files_grouped_filtered_df[["Streams", "Year", "Position"]]

In [107]:
# treemap_df_spec = treemap_df.groupby([ "Year"])["Streams", "Position"].agg('sum')

In [108]:
# treemap_df_spec = treemap_df_spec.reset_index()

In [109]:
# len_v = treemap_df_spec.shape[0]

In [110]:
# treemap_df_spec["side"] = ["right"] * len_v

In [111]:
# treemap_df_spec.columns = ["year", "gas", "miles", "side"]

In [112]:
# treemap_df_spec.to_csv("scatterp.csv", index=False)

In [113]:
# treemap_df_val = treemap_df_spec.pivot(columns=["End Date"], values="Streams", index="Artist")
# treemap_df_val = treemap_df_val.fillna(0)

In [114]:
# treemap_df.melt(columns=["Artist", "End Date"], values="Streams")

In [115]:
# treemap_df_val

In [116]:
# treemap_df_csv = all_files_grouped_filtered_df[["Artist"]]
# treemap_df_csv["Region"] = "ZA"
# treemap_df_csv["State"] = all_files_grouped_filtered_df["Artist"]
# treemap_df_csv["Division"] = all_files_grouped_filtered_df["Artist"]
# treemap_df_csv.to_csv("treemap-region.csv", index=False)

In [117]:
tot_streams_all_files_grouped_filtered_df = all_files_grouped_filtered_df[['Artist', 'Streams']]

In [118]:
tot_streams_all_files_grouped_filtered_df_fin = tot_streams_all_files_grouped_filtered_df.groupby(["Artist"]).sum()

In [119]:
tot_streams_all_files_grouped_filtered_df_fin.reset_index(inplace=True)

In [120]:
tot_streams_all_files_grouped_filtered_df_fin = tot_streams_all_files_grouped_filtered_df_fin.sort_values(by='Streams', ascending=False)

In [121]:
tot_streams_all_files_grouped_filtered_df_fin.head(11)

,Artist,Streams
96,Kabza De Small,15220255
28,Busta 929,12502546
49,De Mthuda,9485271
39,DJ Maphorisa,8412997
141,Mr JazziQ,8288510
223,Young Stunna,5867922
34,DBN Gogo,5447759
145,Musa Keys,4268958
165,Reece Madlisa,4154301
169,Riaan Benad,4014605


In [122]:
top_10_ZA_artists = tot_streams_all_files_grouped_filtered_df_fin["Artist"].tolist()

In [123]:
# top_10_ZA_artists

In [124]:
for artist in top_10_ZA_artists:
    kabza_de_small = all_files_grouped_filtered_df[all_files_grouped_filtered_df["Artist"] == artist]

    kb_racing_bar_df = kabza_de_small[['End Date Aggregated', "Track Name", 'Year', 'Streams']]

    kb_racing_bar_df.columns = ['date', "name", 'category', 'value']
    kb_racing_bar_df = kb_racing_bar_df.groupby(['date', "name", 'category']).sum()
    kb_racing_bar_df.reset_index(inplace=True)
    
    kb_racing_bar_df.to_csv("Classification/topten/_" + artist + "_ZAChartracing_bar.csv", index=False)

### For the bar chart race

In [125]:
len(all_files_grouped_filtered_df)

6305

In [126]:
racing_bar_df = all_files_grouped_filtered_df[['End Date Aggregated', "Artist", 'Year', 'Streams']]

In [127]:
racing_bar_df.columns = ['date', "name", 'category', 'value']

In [128]:
racing_bar_df.to_csv("Classification/JustSouthAfricansOnZAChartracing_bar.csv", index=False)

In [129]:
# len(racing_bar_df["End Date Dt"].unique().tolist())

#### Monthly bar chart race

In [130]:
monthly_racing_bar_df = all_files_grouped_filtered_df[['End Date Dt', "Artist", 'Year', 'Streams']]

# monthly_racing_bar_df.groupby(["Artist", 'End Date Dt']).sum('Streams')

In [131]:
monthly_racing_bar_df["Month"] = monthly_racing_bar_df["End Date Dt"].apply(lambda x : x.month)

<ipython-input-131-a99b727b1bd9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_racing_bar_df["Month"] = monthly_racing_bar_df["End Date Dt"].apply(lambda x : x.month)


In [132]:
monthly_racing_bar_agg_df = monthly_racing_bar_df.groupby(["Artist", "Month", "Year"]).sum("Streams").reset_index()

In [133]:
monthly_racing_bar_agg_df["date"] = monthly_racing_bar_agg_df.apply( lambda x : datetime(int(x["Year"]), int(x["Month"]), 1), axis=1)

In [134]:
monthly_racing_bar_agg_df.drop(["Year"], inplace = True, axis=1)

In [135]:
monthly_racing_bar_fin = pd.DataFrame()
monthly_racing_bar_fin['date'] = monthly_racing_bar_agg_df["date"]
monthly_racing_bar_fin['name'] = monthly_racing_bar_agg_df["Artist"]
monthly_racing_bar_fin['category'] = monthly_racing_bar_agg_df["Month"]
monthly_racing_bar_fin['value'] = monthly_racing_bar_agg_df["Streams"]

In [136]:
monthly_racing_bar_fin.to_csv("Classification/JustSouthAfricansOnZAChartMonthlyracing_bar.csv", index=False)

### animated tree map

In [137]:
# anim_tree_map_df = all_files_grouped_filtered_df[['Year', 'Streams']]

In [138]:
# anim_tree_map_df = anim_tree_map_df.reset_index()

In [139]:
# anim_tree_map_df.drop(["Track Name", "level_2"], axis=1, inplace=True)

In [140]:
# anim_tree_map_df_more = anim_tree_map_df.groupby(['Artist', 'Year']).agg(['sum']).reset_index()

In [141]:
# anim_tree_map_df_more.columns

In [142]:
# anim_tree_map_df_more.columns = ["name", "year", "income"]

In [143]:
# anim_tree_map_df_more['income'] = anim_tree_map_df_more['income'].astype(int)
# anim_tree_map_df_more['year'] = anim_tree_map_df_more['year'].astype(int)

In [144]:
# anim_tree_map_df_more.dtypes

In [145]:
# names = anim_tree_map_df_more.name
# years = anim_tree_map_df_more.year
# streams = anim_tree_map_df_more.income

In [146]:
# anim_tree_map_ls_more = []

# for n in range(0,len(names)):
#     anim_tree_map_ls = [int(years[n]), int(streams[n])]
#     anim_tree_map_dict_more = {}
#     anim_tree_map_dict_more[names[n]] = anim_tree_map_ls
#     anim_tree_map_ls_more.append(anim_tree_map_dict_more)

In [147]:
# anim_tree_map_ls_more

In [148]:
# from collections import defaultdict

# dd = defaultdict(list)

# for d in (anim_tree_map_ls_more): # you can list as many input dicts as you want here
#     for key, value in d.items():
#         dd[key].append(value)

In [149]:
# dict_dd = dict(dd)

In [150]:
# d_keys = list(dict_dd.keys())
# d_values = list(dict_dd.values())
# the_dd = []

# for n in range(0,len(d_keys)):
#     anim_tree_map_dict_more = {}
#     anim_tree_map_dict_more["name"] = d_keys[n]
#     anim_tree_map_dict_more["income"] = d_values[n]
#     anim_tree_map_dict_more["region"] = "ZA"
#     anim_tree_map_dict_more["population"] = d_values[n]
#     anim_tree_map_dict_more["lifeExpectancy"] = d_values[n]
    
    
    
#     the_dd.append(anim_tree_map_dict_more)

In [151]:
# the_dd

In [152]:
# import json
# with open('zaartists.json', 'w') as fp:
#     json.dump(the_dd, fp)